In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf)
rhf.kernel()


print("Atom list:", mol_pyscf.atom)
#print("Basis used:", mol_pyscf.basis)
print("Charge:", mol_pyscf.charge)
print("Spin:", mol_pyscf.spin)
print("Number of electrons:", mol_pyscf.nelectron)

# === ACTIVE SPACE IMPLEMENTATION ===
# Step 1: Define active space parameters
norb_active = 6      # Number of active orbitals
nelec_active = 6     # Number of active electrons

# Step 2: Set up CASSCF to get active space integrals
mycas = mcscf.CASSCF(rhf, norb_active, nelec_active)
mycas.kernel()  # Run CASSCF to get the ground state energy and active space integrals
print("Hartree-Fock Ground State Energy:", rhf.e_tot)
print("CASSCF Ground State Energy:", mycas.e_tot)

# Step 3: Get effective active space integrals
h1e_cas, ecore = mycas.get_h1eff()  # One-electron integrals + core energy
h2e_cas = mycas.get_h2eff()         # Two-electron integrals

print("h2e_cas type:", type(h2e_cas))
print("h2e_cas shape:", h2e_cas.shape) # This result shows a 2D array, symmetric reduced form

norb = norb_active  

# "21" = norb*(norb+1)//2 for norb=6, so s4 symmetry - It was symmetrical reduced. Got into case with no symmetry
h2e_4d = ao2mo.restore(1, h2e_cas, norb) #Check this link (https://pyscf.org/pyscf_api_docs/pyscf.ao2mo.html)

print('Shape of h2e_4d:', h2e_4d.shape)  # Should be (norb, norb, norb, norb)


one_mo = h1e_cas
two_mo = h2e_4d
h2_phys = np.swapaxes(two_mo, 1, 3)  # Ensure the two-electron integrals are in the correct order
core_constant = np.array([ecore])

# Step 5: Continue with your original PennyLane code (unchanged)
H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, h2_phys)
H = qml.jordan_wigner(H_fermionic, tol=1e-16)



#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    #hf_state = qchem.hf_state(active_electrons, qubits)
    hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


converged SCF energy = -1655.1015537099  <S^2> = 6.0386361  2S+1 = 5.0154306
Atom list: Fe 0 0 0; S 0 0 1.826
Charge: 0
Spin: 4
Number of electrons: 42
CASSCF energy = -1655.37432216382
CASCI E = -1655.37432216382  E(CI) = -11.1010881921723  S^2 = 6.0000000
Hartree-Fock Ground State Energy: -1655.101553709904
CASSCF Ground State Energy: -1655.374322163819
h2e_cas type: <class 'numpy.ndarray'>
h2e_cas shape: (21, 21)
Shape of h2e_4d: (6, 6, 6, 6)
Using active space, check if you change the H accordingly
HF state is -1654.8233797961109
Total excitations are 117
The adapt iteration now is 0
The highest operator value is 0.12277410153991322 for operator a⁺(1) a(9)
Highest gradient excitation is [1, 9]
Final updated parameters: [0.18190285]
Final cost: -1654.8289784543383
0: ──X──────────────────────────────────┤  State
1: ──X─╭FermionicSingleExcitation(0.18)─┤  State
2: ──X─├FermionicSingleExcitation(0.18)─┤  State
3: ────├FermionicSingleExcitation(0.18)─┤  State
4: ────├FermionicSingleExc

In [12]:
active_electrons = 6
qubits = 12
singles, doubles = qml.qchem.excitations(active_electrons, qubits)
print('Singles excitations:', singles)
print('Doubles excitations:', doubles)
hf_state = qml.qchem.hf_state(active_electrons, qubits)
print('HF state:', hf_state)
theta = -2.09

dev = qml.device("lightning.qubit", wires=qubits)  
@qml.qnode(dev)
def circuit(hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    qml.SingleExcitation(theta , wires=[0,10])
    return qml.expval(H)

e = circuit(hf_state, qubits, H)
print("Expectation value of the Hamiltonian in the HF state:", e)

Singles excitations: [[0, 6], [0, 8], [0, 10], [1, 7], [1, 9], [1, 11], [2, 6], [2, 8], [2, 10], [3, 7], [3, 9], [3, 11], [4, 6], [4, 8], [4, 10], [5, 7], [5, 9], [5, 11]]
Doubles excitations: [[0, 1, 6, 7], [0, 1, 6, 9], [0, 1, 6, 11], [0, 1, 7, 8], [0, 1, 7, 10], [0, 1, 8, 9], [0, 1, 8, 11], [0, 1, 9, 10], [0, 1, 10, 11], [0, 2, 6, 8], [0, 2, 6, 10], [0, 2, 8, 10], [0, 3, 6, 7], [0, 3, 6, 9], [0, 3, 6, 11], [0, 3, 7, 8], [0, 3, 7, 10], [0, 3, 8, 9], [0, 3, 8, 11], [0, 3, 9, 10], [0, 3, 10, 11], [0, 4, 6, 8], [0, 4, 6, 10], [0, 4, 8, 10], [0, 5, 6, 7], [0, 5, 6, 9], [0, 5, 6, 11], [0, 5, 7, 8], [0, 5, 7, 10], [0, 5, 8, 9], [0, 5, 8, 11], [0, 5, 9, 10], [0, 5, 10, 11], [1, 2, 6, 7], [1, 2, 6, 9], [1, 2, 6, 11], [1, 2, 7, 8], [1, 2, 7, 10], [1, 2, 8, 9], [1, 2, 8, 11], [1, 2, 9, 10], [1, 2, 10, 11], [1, 3, 7, 9], [1, 3, 7, 11], [1, 3, 9, 11], [1, 4, 6, 7], [1, 4, 6, 9], [1, 4, 6, 11], [1, 4, 7, 8], [1, 4, 7, 10], [1, 4, 8, 9], [1, 4, 8, 11], [1, 4, 9, 10], [1, 4, 10, 11], [1, 5, 7, 9], 